# Building a Simple Agent with DLLMForge

This tutorial demonstrates how to build a simple tool-using agent using DLLMForge. You will learn how to configure different LLM providers (Azure OpenAI, OpenAI, Mistral, or Deltares-hosted models), create custom tools, and build an agent that can perform calculations and answer questions about pizza prices.

## Overview

The simple agent tutorial consists of several key components:

1. **Environment Setup**: Configure API keys and credentials for different LLM providers
2. **Tool Creation**: Define custom tools using the DLLMForge `@tool` decorator
3. **Agent Initialization**: Create a SimpleAgent with your chosen LLM provider
4. **Tool Integration**: Add tools to the agent and compile the agent
5. **Query Processing**: Test the agent with various queries and observe tool routing
6. **Provider Switching**: Change between different LLM providers without code changes

## Prerequisites

1. Python environment with the project requirements installed
2. A `.env` file in your project root with provider credentials (see below)
3. Optional: IPython/Jupyter if you want to display the LangGraph diagram

### Environment Setup

Create or update your `.env` with the variables for the providers you plan to use.

**Azure OpenAI (default in DLLMForge examples):**
```bash
AZURE_OPENAI_ENDPOINT=https://your-azure-endpoint
AZURE_OPENAI_API_KEY=your_azure_openai_api_key
AZURE_OPENAI_DEPLOYMENT_NAME=your_deployment_name
AZURE_OPENAI_API_VERSION=2024-12-01-preview
```

**OpenAI:**
```bash
OPENAI_API_KEY=your_openai_api_key
OPENAI_MODEL_NAME=gpt-4o  # or gpt-4o, etc.
```

**Mistral:**
```bash
MISTRAL_API_KEY=your_mistral_api_key
MISTRAL_MODEL_NAME=mistral-large-latest
```

**Deltares-hosted (no API key; requires Deltares network/VPN):**
```bash
# No keys required; you will specify base_url and model at runtime
DELTARES_BASE_URL=https://chat-api.directory.intra
DELTARES_MODEL_NAME=llama3.1:70b
```

## Step 1: Import Required Modules

Start by loading environment variables and importing all necessary components.

In [1]:
# Load environment variables from .env file for API keys and endpoints
from dotenv import load_dotenv
load_dotenv()

import os
import sys
from pathlib import Path

# Import dllmforge simple agent and tool decorator
from dllmforge.agent_core import SimpleAgent, tool

## Step 2: Define Basic Math Tools

Create custom tools using the `@tool` decorator. These tools will be available to the agent for performing calculations.

In [2]:
@tool
def add(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b


@tool
def subtract(a: float, b: float) -> float:
    """Subtract two numbers from each other."""
    return a - b


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


@tool
def divide(a: float, b: float) -> float:
    """Divide two numbers."""
    return a / b

print("✅ Math tools defined: add, subtract, multiply, divide")

INFO:dllmforge.agent_core:Registering DLLMForge tool: add
INFO:dllmforge.agent_core:Registering DLLMForge tool: subtract
INFO:dllmforge.agent_core:Registering DLLMForge tool: multiply
INFO:dllmforge.agent_core:Registering DLLMForge tool: divide
INFO:dllmforge.agent_core:Registering DLLMForge tool: subtract
INFO:dllmforge.agent_core:Registering DLLMForge tool: multiply
INFO:dllmforge.agent_core:Registering DLLMForge tool: divide


✅ Math tools defined: add, subtract, multiply, divide


## Step 3: Define Pizza Pricing Tool

Create a domain-specific tool that retrieves pizza prices.

In [3]:
@tool
def get_pizza_price(pizza_type: str) -> float:
    """Get the price of a pizza type."""
    prices = {
        "margherita": 12.99,
        "pepperoni": 15.99,
        "vegetarian": 14.99,
        "supreme": 17.99
    }
    return prices.get(pizza_type.lower(), 10.99)

print("✅ Pizza pricing tool defined")
print("Available pizzas:", {"margherita": 12.99, "pepperoni": 15.99, "vegetarian": 14.99, "supreme": 17.99})

INFO:dllmforge.agent_core:Registering DLLMForge tool: get_pizza_price


✅ Pizza pricing tool defined
Available pizzas: {'margherita': 12.99, 'pepperoni': 15.99, 'vegetarian': 14.99, 'supreme': 17.99}


## Step 4: Define LLM-Powered Summary Tool

Create a tool that uses an LLM to generate conversational summaries of results. This demonstrates how tools can call LLMs internally.

In [4]:
# Provider switch for the summary tool: 'azure' or 'deltares'
PROVIDER = (os.getenv("PIZZA_LLM_PROVIDER") or "azure").lower()
deltares_available = False  # toggled true after connectivity check
shared_llm = None  # Reused when using Deltares for both agent and summary


@tool
def make_summary(question: str, result: str) -> str:
    """Use the configured LLM to create a concise, conversational summary.

    Args:
        question: The original user question.
        result: The computed or retrieved result to summarize.

    Returns:
        A short, user-friendly summary generated by the LLM.
    """
    try:
        from langchain_core.messages import SystemMessage, HumanMessage

        if PROVIDER == "deltares" and deltares_available:
            # Use a shared Deltares LLM instance
            global shared_llm
            if shared_llm is None:
                from dllmforge.LLMs.Deltares_LLMs import DeltaresOllamaLLM
                base_url = os.getenv("DELTARES_BASE_URL", "https://chat-api.directory.intra")
                model_name = os.getenv("DELTARES_MODEL_NAME", "llama3.1:70b")
                shared_llm = DeltaresOllamaLLM(base_url=base_url, model_name=model_name)
            invoke_fn = shared_llm.invoke
        else:
            # Default to Azure OpenAI via DLLMForge's LangchainAPI
            from dllmforge.langchain_api import LangchainAPI
            llm_api = LangchainAPI()  # defaults use env to configure Azure
            invoke_fn = llm_api.llm.invoke

        messages = [
            SystemMessage(
                content=(
                    "You are a helpful assistant. Create a concise, friendly summary of the provided result "
                    "in the context of the question. Mention all of the tools that you used"
                )
            ),
            HumanMessage(
                content=(
                    f"Question:\n{question}\n\nResult:\n{result}\n\nPlease return a brief, conversational summary (1-3 sentences)."
                )
            ),
        ]
        response = invoke_fn(messages)
        return getattr(response, "content", str(response))
    except Exception as e:
        return f"Could not generate summary: {e}"

print("✅ Summary tool defined (uses LLM internally)")

INFO:dllmforge.agent_core:Registering DLLMForge tool: make_summary


✅ Summary tool defined (uses LLM internally)


## Step 5: Create the Agent

Initialize a SimpleAgent with clear instructions. The agent will use the configured LLM provider (Azure OpenAI by default, or Deltares if configured).

In [5]:
# Create agent based on provider configuration
if PROVIDER == "deltares":
    # Build and reuse a Deltares LLM for both routing and summarisation
    from dllmforge.LLMs.Deltares_LLMs import DeltaresOllamaLLM
    base_url = os.getenv("DELTARES_BASE_URL", "https://chat-api.directory.intra")
    model_name = os.getenv("DELTARES_MODEL_NAME", "llama3.1:70b")
    shared_llm = DeltaresOllamaLLM(base_url=base_url, model_name=model_name)

    # Connectivity check with a tiny ping; fallback to Azure if it fails
    try:
        from langchain_core.messages import HumanMessage
        _ = shared_llm.invoke([HumanMessage(content="ping")])
        deltares_available = True
        print("✅ Connected to Deltares LLM")
    except Exception as e:
        print(f"⚠️ Deltares LLM not reachable ({e}). Falling back to Azure for routing and summary.")

    if deltares_available:
        routing_system = (
            "You are a helpful assistant that can do math and tell you about pizza prices."
            " When you need a tool, respond ONLY with a JSON object like {\"tool\": \"<name>\", \"args\": {...}}."
            " Use exact tool names: add, multiply, divide, subtract, get_pizza_price, make_summary."
        )
        agent = SimpleAgent(
            routing_system,
            temperature=0.1,
            llm=shared_llm,
            enable_text_tool_routing=True,
            max_tool_iterations=4,
        )
    else:
        agent = SimpleAgent(
            "You are a helpful assistant that can do math and tell you about pizza prices. Only use the tools, do not try to do maths in your head.",
            temperature=0.1
        )
else:
    agent = SimpleAgent(
        "You are a helpful assistant that can do math and tell you about pizza prices. Only use the tools, do not try to do maths in your head.",
        temperature=0.1
    )
    print("✅ Agent created with Azure OpenAI (default)")

print("\n📋 System Message:")
print(agent.system_message)

INFO:dllmforge.agent_core:Simple agent initialized


✅ Agent created with Azure OpenAI (default)

📋 System Message:
You are a helpful assistant that can do math and tell you about pizza prices. Only use the tools, do not try to do maths in your head.


## Step 6: Add Tools to the Agent

Register all the tools we've created with the agent.

In [6]:
agent.add_tool(make_summary)
agent.add_tool(divide)
agent.add_tool(multiply)
agent.add_tool(add)
agent.add_tool(subtract)
agent.add_tool(get_pizza_price)

print("✅ All tools added to agent:")
print("  • Math tools: add, subtract, multiply, divide")
print("  • Domain tools: get_pizza_price")
print("  • LLM tools: make_summary")

INFO:dllmforge.agent_core:Added tool: make_summary
INFO:dllmforge.agent_core:Added tool: divide
INFO:dllmforge.agent_core:Added tool: multiply
INFO:dllmforge.agent_core:Added tool: add
INFO:dllmforge.agent_core:Added tool: subtract
INFO:dllmforge.agent_core:Added tool: get_pizza_price
INFO:dllmforge.agent_core:Added tool: divide
INFO:dllmforge.agent_core:Added tool: multiply
INFO:dllmforge.agent_core:Added tool: add
INFO:dllmforge.agent_core:Added tool: subtract
INFO:dllmforge.agent_core:Added tool: get_pizza_price


✅ All tools added to agent:
  • Math tools: add, subtract, multiply, divide
  • Domain tools: get_pizza_price
  • LLM tools: make_summary


## Step 7: Compile the Agent

Compile the agent workflow. This creates the LangGraph workflow that handles tool routing and execution.

In [7]:
agent.compile()

print("✅ Agent compiled and ready to use!")
print("\n" + "=" * 60)
print("🍕 PIZZA AGENT READY")
print("=" * 60)

INFO:dllmforge.agent_core:Added node: agent
INFO:dllmforge.agent_core:Added node: tools
INFO:dllmforge.agent_core:Added edge: __start__ -> agent
INFO:dllmforge.agent_core:Added conditional edge from: agent
INFO:dllmforge.agent_core:Added edge: tools -> agent
INFO:dllmforge.agent_core:Simple workflow created with human interaction support
INFO:dllmforge.agent_core:Added node: tools
INFO:dllmforge.agent_core:Added edge: __start__ -> agent
INFO:dllmforge.agent_core:Added conditional edge from: agent
INFO:dllmforge.agent_core:Added edge: tools -> agent
INFO:dllmforge.agent_core:Simple workflow created with human interaction support
INFO:dllmforge.agent_core:Workflow compiled successfully
INFO:dllmforge.agent_core:Workflow compiled successfully


✅ Agent compiled and ready to use!

🍕 PIZZA AGENT READY


## Testing the Agent

Now let's test the agent with various queries to see how it uses tools to solve problems.

### Test Case 1: Simple Pizza Calculation

This query asks about the total price of two pizzas. The agent should:
1. Use `get_pizza_price` to get the pepperoni price (15.99)
2. Use `get_pizza_price` to get the margherita price (12.99)
3. Use `add` to calculate the total (28.98)
4. Use `make_summary` to provide a friendly response

In [8]:
query1 = "What is the total price of a pepperoni pizza and a margherita pizza?"
print(f"🧪 TEST 1: {query1}")
print("-" * 60)
agent.process_query(query1, stream=True)

🧪 TEST 1: What is the total price of a pepperoni pizza and a margherita pizza?
------------------------------------------------------------

PROCESSING: What is the total price of a pepperoni pizza and a margherita pizza?

================================ Human Message =================================

What is the total price of a pepperoni pizza and a margherita pizza?



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_pizza_price (call_0HeoftdYERu5EstXAEzExwXt)
 Call ID: call_0HeoftdYERu5EstXAEzExwXt
  Args:
    pizza_type: pepperoni
  get_pizza_price (call_TenETx7n8ighc7Emn5oodwvE)
 Call ID: call_TenETx7n8ighc7Emn5oodwvE
  Args:
    pizza_type: margherita

================================= Tool Message =================================
Name: get_pizza_price

12.99



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  add (call_6N1br5ZE0Q0dY213fGrwEgro)
 Call ID: call_6N1br5ZE0Q0dY213fGrwEgro
  Args:
    a: 15.99
    b: 12.99

================================= Tool Message =================================
Name: add

28.98



...

================================== Ai Message ==================================
Tool Calls:
  make_summary (call_JzlRzdkED2NzjjG3HG6OCjUD)
 Call ID: call_JzlRzdkED2NzjjG3HG6OCjUD
  Args:
    question: What is the total price of a pepperoni pizza and a margherita pizza?
    result: The total price for a pepperoni pizza ($15.99) and a margherita pizza ($12.99) is $28.98.



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================= Tool Message =================================
Name: make_summary

If you order a pepperoni pizza and a margherita pizza together, the total comes to $28.98. I used the menu prices for both pizzas to calculate this.



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================

If you order a pepperoni pizza and a margherita pizza together, the total comes to $28.98. I used the menu prices for both pizzas to calculate this.



### Test Case 2: Complex Fraction Calculation

This query involves fractions and cost-sharing. The agent should:
1. Get the pepperoni price (15.99)
2. Get the margherita price (12.99)
3. Calculate 2/3 of the pepperoni price (friend's share)
4. Calculate 1/2 of the margherita price (friend's share)
5. Add the two amounts to get the total the friend owes
6. Generate a conversational summary with the Tikkie amount

In [9]:
query2 = "My friend ate 2/3 of a pepperoni pizza and I ate 1/2 of a margherita pizza and I paid for both pizzas. How much should I Tikkie her for her share?"
print(f"🧪 TEST 2: {query2}")
print("-" * 60)
agent.process_query(query2, stream=True)

🧪 TEST 2: My friend ate 2/3 of a pepperoni pizza and I ate 1/2 of a margherita pizza and I paid for both pizzas. How much should I Tikkie her for her share?
------------------------------------------------------------

PROCESSING: My friend ate 2/3 of a pepperoni pizza and I ate 1/2 of a margherita pizza and I paid for both pizzas. How much should I Tikkie her for her share?

================================ Human Message =================================

My friend ate 2/3 of a pepperoni pizza and I ate 1/2 of a margherita pizza and I paid for both pizzas. How much should I Tikkie her for her share?



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_pizza_price (call_siF6j32Yozu6fxJ5hrplcOLp)
 Call ID: call_siF6j32Yozu6fxJ5hrplcOLp
  Args:
    pizza_type: pepperoni
  get_pizza_price (call_VTi4gId6BLAEznFHEFZ0uhMO)
 Call ID: call_VTi4gId6BLAEznFHEFZ0uhMO
  Args:
    pizza_type: margherita

================================= Tool Message =================================
Name: get_pizza_price

12.99



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  multiply (call_DuJsxrnUadjkDZ81pdYNw9hm)
 Call ID: call_DuJsxrnUadjkDZ81pdYNw9hm
  Args:
    a: 15.99
    b: 0.6667
  multiply (call_oPfUZaFaRHgTlZj9yh6soT2G)
 Call ID: call_oPfUZaFaRHgTlZj9yh6soT2G
  Args:
    a: 12.99
    b: 0.5

================================= Tool Message =================================
Name: multiply

6.495



...

================================== Ai Message ==================================
Tool Calls:
  make_summary (call_SixuHoIn7VAXkq0vs9mYpt8T)
 Call ID: call_SixuHoIn7VAXkq0vs9mYpt8T
  Args:
    question: How much should I Tikkie my friend for her share if she ate 2/3 of a pepperoni pizza and I ate 1/2 of a margherita pizza, and I paid for both?
    result: You should Tikkie your friend €8.58 for her share.



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================= Tool Message =================================
Name: make_summary

You should Tikkie your friend €8.58 for her share of the pizzas. I used basic fraction math to split the costs based on how much each of you ate. Let me know if you need a breakdown of the calculation!



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================

You should Tikkie your friend €8.58 for her share of the pizzas. This splits the cost fairly based on how much each of you ate. Let me know if you want to see the calculation details!



### Test Case 3: Multiple Pizza Types

Let's try a query with different pizza types.

In [10]:
query3 = "If I order 2 supreme pizzas and 1 vegetarian pizza, what's the total cost?"
print(f"🧪 TEST 3: {query3}")
print("-" * 60)
agent.process_query(query3, stream=True)

🧪 TEST 3: If I order 2 supreme pizzas and 1 vegetarian pizza, what's the total cost?
------------------------------------------------------------

PROCESSING: If I order 2 supreme pizzas and 1 vegetarian pizza, what's the total cost?

================================ Human Message =================================

If I order 2 supreme pizzas and 1 vegetarian pizza, what's the total cost?



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_pizza_price (call_npcopaVRmLWqtZZxPrCtuSQk)
 Call ID: call_npcopaVRmLWqtZZxPrCtuSQk
  Args:
    pizza_type: supreme
  get_pizza_price (call_vQxrqKEq6TzjuwSsa6aKUQHF)
 Call ID: call_vQxrqKEq6TzjuwSsa6aKUQHF
  Args:
    pizza_type: vegetarian

================================= Tool Message =================================
Name: get_pizza_price

14.99



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  multiply (call_xfxyMqJgLOn0vnJaJXGmkagC)
 Call ID: call_xfxyMqJgLOn0vnJaJXGmkagC
  Args:
    a: 2
    b: 17.99
  multiply (call_uSSCY2numUduIUSOUoRhmrHe)
 Call ID: call_uSSCY2numUduIUSOUoRhmrHe
  Args:
    a: 1
    b: 14.99

================================= Tool Message =================================
Name: multiply

14.99



...

================================== Ai Message ==================================
Tool Calls:
  make_summary (call_0z731pOhHBT2ZTHCBWk7fBhV)
 Call ID: call_0z731pOhHBT2ZTHCBWk7fBhV
  Args:
    question: If I order 2 supreme pizzas and 1 vegetarian pizza, what's the total cost?
    result: The total cost for 2 supreme pizzas and 1 vegetarian pizza is $50.97.



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================= Tool Message =================================
Name: make_summary

If you order 2 supreme pizzas and 1 vegetarian pizza, your total comes to $50.97. I used the menu prices to calculate this for you!



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================

If you order 2 supreme pizzas and 1 vegetarian pizza, your total comes to $50.97. I used the menu prices to calculate this for you!



### Test Case 4: Simple Math

Test the agent's pure calculation abilities.

In [11]:
query4 = "What is 25.5 divided by 3, then multiplied by 2?"
print(f"🧪 TEST 4: {query4}")
print("-" * 60)
agent.process_query(query4, stream=True)

🧪 TEST 4: What is 25.5 divided by 3, then multiplied by 2?
------------------------------------------------------------

PROCESSING: What is 25.5 divided by 3, then multiplied by 2?

================================ Human Message =================================

What is 25.5 divided by 3, then multiplied by 2?



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  divide (call_Q8rTaVfMi0j0eQHAFo4FITV9)
 Call ID: call_Q8rTaVfMi0j0eQHAFo4FITV9
  Args:
    a: 25.5
    b: 3

================================= Tool Message =================================
Name: divide

8.5



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  multiply (call_1sgNK50ZYaRUe5igm24Dqbf4)
 Call ID: call_1sgNK50ZYaRUe5igm24Dqbf4
  Args:
    a: 8.5
    b: 2

================================= Tool Message =================================
Name: multiply

17.0



...

================================== Ai Message ==================================
Tool Calls:
  make_summary (call_PwFG1OkWRoAo3hL6AUwdKblS)
 Call ID: call_PwFG1OkWRoAo3hL6AUwdKblS
  Args:
    question: What is 25.5 divided by 3, then multiplied by 2?
    result: The answer is 17.



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================= Tool Message =================================
Name: make_summary

To solve this, I first divided 25.5 by 3 to get 8.5, then multiplied that by 2 to get 17. I used basic arithmetic operations (division and multiplication) to find the answer. So, 25.5 divided by 3 and then multiplied by 2 equals 17!



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================

25.5 divided by 3 is 8.5, and when you multiply that by 2, you get 17. So, the answer is 17!



## Try Your Own Query

Now it's your turn! Try asking the agent your own questions about pizza prices or calculations.

In [12]:
# Try your own query here
your_query = "How much would 3 margherita pizzas and 2 pepperoni pizzas cost?"
print(f"Your query: {your_query}")
print("-" * 60)
agent.process_query(your_query, stream=True)

Your query: How much would 3 margherita pizzas and 2 pepperoni pizzas cost?
------------------------------------------------------------

PROCESSING: How much would 3 margherita pizzas and 2 pepperoni pizzas cost?

================================ Human Message =================================

How much would 3 margherita pizzas and 2 pepperoni pizzas cost?



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_pizza_price (call_CmcxkBKM82Yt8eFNa5inddmj)
 Call ID: call_CmcxkBKM82Yt8eFNa5inddmj
  Args:
    pizza_type: margherita
  get_pizza_price (call_a1Bi5LUIgYHhIHZt3WlqI65e)
 Call ID: call_a1Bi5LUIgYHhIHZt3WlqI65e
  Args:
    pizza_type: pepperoni

================================= Tool Message =================================
Name: get_pizza_price

15.99



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  multiply (call_g1hGBLeVQCBmlbOqXz6pBXCu)
 Call ID: call_g1hGBLeVQCBmlbOqXz6pBXCu
  Args:
    a: 3
    b: 12.99
  multiply (call_Nd4J8RYfCvJwX5xglvjkxOmP)
 Call ID: call_Nd4J8RYfCvJwX5xglvjkxOmP
  Args:
    a: 2
    b: 15.99

================================= Tool Message =================================
Name: multiply

31.98



...

================================== Ai Message ==================================
Tool Calls:
  make_summary (call_6g4Htdcy5gCbrWSSDYnqhPA0)
 Call ID: call_6g4Htdcy5gCbrWSSDYnqhPA0
  Args:
    question: How much would 3 margherita pizzas and 2 pepperoni pizzas cost?
    result: The total cost for 3 margherita pizzas and 2 pepperoni pizzas is $70.95.



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================= Tool Message =================================
Name: make_summary

Sure! The total for 3 margherita pizzas and 2 pepperoni pizzas comes to $70.95. I used the provided pricing information to calculate this for you.



INFO:httpx:HTTP Request: POST https://openaicoastal.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


================================== Ai Message ==================================

The total for 3 margherita pizzas and 2 pepperoni pizzas comes to $70.95. Let me know if you need prices for other combinations!



## Key Takeaways

✅ **Simple Tool Creation**: Use the `@tool` decorator to create custom tools with minimal boilerplate

✅ **Flexible LLM Providers**: Switch between Azure OpenAI, OpenAI, Mistral, or Deltares-hosted models easily

✅ **Automatic Tool Routing**: The agent automatically decides which tools to use based on the query

✅ **Composable Tools**: Tools can call LLMs internally (like the `make_summary` tool)

✅ **Streaming Output**: Use `stream=True` to see tool calls and results in real-time

✅ **One-Line Compilation**: Simple `.compile()` call creates a complete LangGraph workflow

## Next Steps

- Add more domain-specific tools for your use case
- Explore the advanced agent tutorial for conditional routing and specialized nodes
- Try different LLM providers to compare cost, latency, and quality
- Build RAG systems with document retrieval tools
- Create multi-agent systems for complex workflows

This tutorial provides a solid foundation for building tool-using agents with DLLMForge! 🍕✨